In [1]:
import matplotlib as mpl
#mpl.use('Agg')
import random as rn
import fiona
import pandas as pd
import numpy as np
from random import randint as rint
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def dist(i,j):
    return np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 )



fp = fiona.open("/home/dario/Documentos/TR/redCDMX/VialidadOSM.shp")
calles = {}
siq  = [u'highway', u'barrier', u'id', u'nombre', u'osm_id', u'other_tags', u'sentido', u'tipo', u'z_order']
for feat in fp:
    calles[ feat['id'] ] = {}
    calles[ feat['id'] ]['type'] = feat['geometry']['type']
    calles[ feat['id'] ]['coords'] = feat['geometry']['coordinates']
    for key, val in feat['properties'].items():
        if key in siq:
            calles[ feat['id'] ][key] = val
            calles[ feat['id'] ][key] = val

            
fp.close()            

ntwk = pd.DataFrame(calles).transpose()
ntwk.index = ntwk['id']
ntwk = ntwk.drop(['id'], axis=1)


i=0
GN = nx.DiGraph(weighted=True)
for row, col in ntwk.iterrows():
    p = ntwk.loc[row]['coords']
    lp = len(p)       
    GN.add_path( p, name=ntwk.loc[row]['nombre'],
                vialidad = ntwk.loc[row]['highway'], osm_id = ntwk.loc[row]['osm_id'],
                sentido = ntwk.loc[row]['sentido'], tipo=ntwk.loc[row]['tipo']
               )
    i = lp + i
    


    

GG = nx.Graph(GN)
gnc = sorted(nx.connected_components(GG), key = len, reverse=True)[0]
gc = GN.subgraph(gnc)
gnfc = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[0]
gfc = GN.subgraph(gnfc)
gfc1 = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[1]
gf1 = GN.subgraph(gfc1)
gfc = nx.DiGraph(gfc)


In [2]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [3]:
ini = pd.read_csv('/home/dario/Escritorio/inness_csv/inness_fastest_I.csv')
isi = pd.read_csv('/home/dario/Escritorio/inness_csv/inness_shortest_I.csv')
#inr = pd.read_csv('/home/dario/Escritorio/inness_csv/inness_fastest_rho_A.csv')
#isr = pd.read_csv('/home/dario/Escritorio/inness_csv/inness_shortest_rho_A.csv')


In [4]:
ls = []
for xx in [ini, isi,]: # inr, isr]:
    xx = xx.loc[ xx['city'] == 'Mexico_City'].drop('Unnamed: 0', axis=1)
    print len(xx)
    ls += [xx]

3029
3184


In [83]:
a = 19.4006, -99.1289
b = 19.3914, -99.1291
dist(a,b)

0.0092021736562615825

In [5]:
df = ls[1]
n4 = [x for x in gfc.nodes() if nx.degree(gfc, x) > 3]
c = (19.432608, -99.133209)
unk = 0.0092
df['or'] = df.apply(lambda x:  (c[0] + x['radius']*unk*np.cos(np.radians(x['origin_angle'])),
                    c[1] + x['radius']*unk*np.sin(np.radians(x['origin_angle']))), axis=int(1))
df['de'] = df.apply(lambda x: (c[0] + x['radius']*unk*np.cos(np.radians(x['destination_angle'])),
                    c[1] + x['radius']*unk*np.sin(np.radians(x['destination_angle']))), axis=int(1))

    


In [6]:
def dif(a, b):
    return np.abs(a[0]-b[0])+np.abs(a[1]-b[1])


def closst(a):
    dsts = [dist(a,x) for x in n4]
    for y in n4:
        if dist(a,y) == np.min(dsts):
            return y
            break
        

In [15]:
def closst(a):
    d = dist(a, n4[49])
    for y in n4:
        if dist(a,y) < d:
            d = dist(a,y)
            z = y
    return z


def closst(a):
    d = dist(a, n4[94])
    for y in n4:
        yy = (y[1], y[0])
        if dist(a,yy) < d:
            d = dist(a,yy)
            z = y
    return z

In [ ]:
df['oor'] = df.apply(lambda x: closst(x['or']), axis=int(1))
df['dde'] = df.apply(lambda x: closst(x['de']), axis=int(1))

In [ ]:

for edge in gfc.edges():
    gfc[edge[0]][edge[1]]['dist'] = dist(*edge)
    

    





In [ ]:
df['sp'] = df.apply(lambda x: nx.dijkstra_path(gfc, x['oor'], x['dde'], weight='dist'), axis=int(1))    


In [ ]:
dinn = {}
dinn = dict.fromkeys(n4, 0.0)
i= 0
for xxx in list(df['sp']):
    for yy in xxx:
        dinn[yy] = df['normalized_inness']
        
        
import pickle

with open('/home/dario/Escritorio/dictinn.py', 'wb') as fp:
    pickle.dump(dinn, fp)

In [ ]:
def tam(n):
    if n < 3:
        return 1
    elif n == 3:
        return 1
    elif n == 4:
        return 2
    elif n == 5: 
        return 3
    elif n > 5:
        return 3
        
        
gradost['tam'] = gradost['gr'].apply(lambda l: tam(l))


In [ ]:

##### ESTO LO METISTE A BORROMEO PARA OBTENER DICCIONARIOS CON CENTRALIDADES


import random as rn
import fiona
import pandas as pd
import numpy as np
from random import randint as rint
import networkx as nx
from heapq import heappush, heappop
from itertools import count


def dist(i,j):
    return np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 )


# helpers for betweenness centrality

def _single_source_shortest_path_basic(G, s):
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    D[s] = 0
    Q = [s]
    while Q:   # use BFS to find shortest paths
        v = Q.pop(0)
        S.append(v)
        Dv = D[v]
        sigmav = sigma[v]
        for w in G[v]:
            if w not in D:
                Q.append(w)
                D[w] = Dv + 1
            if D[w] == Dv + 1:   # this is a shortest path, count paths
                sigma[w] += sigmav
                P[w].append(v)  # predecessors
    return S, P, sigma


def _single_source_dijkstra_path_basic(G, s, weight='weight'):
    # modified from Eppstein
    S = []
    P = {}
#    nod = [x for x in G.nodes() if nx.degree(G,x) > 3]
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    push = heappush
    pop = heappop
    seen = {s: 0}
    c = count()
    Q = []   # use Q as heap with (distance,node id) tuples
    push(Q, (0, next(c), s, s))
    while Q:
        (dist, _, pred, v) = pop(Q)
        if v in D:
            continue  # already searched this node.
        sigma[v] += sigma[pred]  # count paths
        S.append(v)
        D[v] = dist
        for w, edgedata in G[v].items():
            vw_dist = dist + edgedata.get(weight, 1)
            if w not in D and (w not in seen or vw_dist < seen[w]):
                seen[w] = vw_dist
                push(Q, (vw_dist, next(c), v, w))
                sigma[w] = 0.0
                P[w] = [v]
            elif vw_dist == seen[w]:  # handle equal paths
                sigma[w] += sigma[v]
                P[w].append(v)
    return S, P, sigma


def _accumulate_basic(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w]
    return betweenness


def _accumulate_endpoints(betweenness, S, P, sigma, s):
    betweenness[s] += len(S) - 1
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff
        if w != s:
            betweenness[w] += delta[w] + 1
    return betweenness


def _accumulate_edges(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            c = sigma[v] * coeff
            if (v, w) not in betweenness:
                betweenness[(w, v)] += c
            else:
                betweenness[(v, w)] += c
            delta[v] += c
        if w != s:
            betweenness[w] += delta[w]
    return betweenness


def _rescale(betweenness, n, normalized, directed=False, k=None):
    if normalized is True:
        if n <= 2:
            scale = None  # no normalization b=0 for all nodes
        else:
            scale = 1.0 / ((n - 1) * (n - 2))
    else:  # rescale by 2 for undirected graphs
        if not directed:
            scale = 1.0 / 2.0
        else:
            scale = None
    if scale is not None:
        if k is not None:
            scale = scale * n / k
        for v in betweenness:
            betweenness[v] *= scale
    return betweenness


def _rescale_e(betweenness, n, normalized, directed=False, k=None):
    if normalized is True:
        if n <= 1:
            scale = None  # no normalization b=0 for all nodes
        else:
            scale = 1.0 / (n * (n - 1))
    else:  # rescale by 2 for undirected graphs
        if not directed:
            scale = 1.0 / 2.0
        else:
            scale = None
    if scale is not None:
        if k is not None:
            scale = scale * n / k
        for v in betweenness:
            betweenness[v] *= scale
    return betweenness
            
fp = fiona.open("/home/dario/trafico/redCDMX/VialidadOSM.shp")
calles = {}
siq  = [u'highway', u'barrier', u'id', u'nombre', u'osm_id', u'other_tags', u'sentido', u'tipo', u'z_order']
for feat in fp:
    calles[ feat['id'] ] = {}
    calles[ feat['id'] ]['type'] = feat['geometry']['type']
    calles[ feat['id'] ]['coords'] = feat['geometry']['coordinates']
    for key, val in feat['properties'].items():
        if key in siq:
            calles[ feat['id'] ][key] = val
            calles[ feat['id'] ][key] = val


fp.close()

ntwk = pd.DataFrame(calles).transpose()
ntwk.index = ntwk['id']
ntwk = ntwk.drop(['id'], axis=1)


i=0
GN = nx.DiGraph(weighted=True)
for row, col in ntwk.iterrows():
    p = ntwk.loc[row]['coords']
    lp = len(p)
    GN.add_path( p, name=ntwk.loc[row]['nombre'],
                vialidad = ntwk.loc[row]['highway'], osm_id = ntwk.loc[row]['osm_id'],
                sentido = ntwk.loc[row]['sentido'], tipo=ntwk.loc[row]['tipo']
               )
    i = lp + i
    
    
gnfc = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[0]
gfc = GN.subgraph(gnfc)
for edge in gfc.edges():
    gfc[edge[0]][edge[1]]['dist'] = dist(*edge)



def betw_c(G, k=None, normalized=True, weight=None,
                           endpoints=False,
                           seed=None):
    betweenness = dict.fromkeys(G, 0.0)  # b[v]=0 for v in G
    if k is None:
        nodes = G
    else:
        rn.seed(seed)
        nodes = rn.sample([x for x in G.nodes() if nx.degree(GN,x) > 3], k)
    aaa = len(nodes)
    for s in nodes[]:
        # single source shortest paths
        if weight is None:  # use BFS
            S, P, sigma = _single_source_shortest_path_basic(G, s)
        else:  # use Dijkstra's algorithm
            S, P, sigma = _single_source_dijkstra_path_basic(G, s, weight)
        # accumulation
        if endpoints:
            betweenness = _accumulate_endpoints(betweenness, S, P, sigma, s)
        else:
            betweenness = _accumulate_basic(betweenness, S, P, sigma, s)
    # rescaling
    betweenness = _rescale(betweenness, len(G),
                           normalized=normalized,
                           directed=G.is_directed(),
                           k=k)
    return betweenness




dbc = betw_c(gfc, k=20000, normalized=True, weight='dist',
                           endpoints=False,
                           seed=None)


import pickle
with open('dic_betw_cent3.py', 'wb') as fp:
    pickle.dump(dbc, fp)
    



try:
    df = pd.DataFrame(dbc)
    df.to_csv('dic_betw_cent3.csv')
except:
    pass




In [ ]:
### Closeness

import random as rn
import fiona
import pandas as pd
import numpy as np
from random import randint as rint
import networkx as nx
import functools
from collections import deque
from heapq import heappush, heappop
from itertools import count
import networkx as nx
from networkx.utils import generate_unique_node


def dist(i,j):
    return np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 )




fp = fiona.open("/home/dario/trafico/redCDMX/VialidadOSM.shp")
calles = {}
siq  = [u'highway', u'barrier', u'id', u'nombre', u'osm_id', u'other_tags', u'sentido', u'tipo', u'z_order']
for feat in fp:
    calles[ feat['id'] ] = {}
    calles[ feat['id'] ]['type'] = feat['geometry']['type']
    calles[ feat['id'] ]['coords'] = feat['geometry']['coordinates']
    for key, val in feat['properties'].items():
        if key in siq:
            calles[ feat['id'] ][key] = val
            calles[ feat['id'] ][key] = val


fp.close()

ntwk = pd.DataFrame(calles).transpose()
ntwk.index = ntwk['id']
ntwk = ntwk.drop(['id'], axis=1)


i=0
GN = nx.DiGraph(weighted=True)
for row, col in ntwk.iterrows():
    p = ntwk.loc[row]['coords']
    lp = len(p)
    GN.add_path( p, name=ntwk.loc[row]['nombre'],
                vialidad = ntwk.loc[row]['highway'], osm_id = ntwk.loc[row]['osm_id'],
                sentido = ntwk.loc[row]['sentido'], tipo=ntwk.loc[row]['tipo']
               )
    i = lp + i
    
    
gnfc = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[0]
gfc = GN.subgraph(gnfc)

for edge in gfc.edges():
    gfc[edge[0]][edge[1]]['dist'] = dist(*edge)



def _dijkstra(G, source, get_weight, pred=None, paths=None, cutoff=None,
              target=None):
    G_succ = G.succ if G.is_directed() else G.adj
    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    seen = {source: 0}
    c = count()
    fringe = []  # use heapq with (distance,label) tuples
    push(fringe, (0, next(c), source))
    while fringe:
        (d, _, v) = pop(fringe)
        if v in dist:
            continue  # already searched this node.
        dist[v] = d
        if v == target:
            break
        for u, e in G_succ[v].items():
            cost = get_weight(v, u, e)
            if cost is None:
                continue
            vu_dist = dist[v] + get_weight(v, u, e)
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                if vu_dist < dist[u]:
                    raise ValueError('Contradictory paths found:',
                                     'negative weights?')
            elif u not in seen or vu_dist < seen[u]:
                seen[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
                if paths is not None:
                    paths[u] = paths[v] + [u]
                if pred is not None:
                    pred[u] = [v]
            elif vu_dist == seen[u]:
                if pred is not None:
                    pred[u].append(v)

    if paths is not None:
        return (dist, paths)
    if pred is not None:
        return (pred, dist)
    return dist


def single_source_dijkstra_path_length_g(G, source, target, cutoff=None,
                                       weight='weight'):
    if G.is_multigraph():
        get_weight = lambda u, v, data: min(
            eattr.get(weight, 1) for eattr in data.values())
    else:
        get_weight = lambda u, v, data: data.get(weight, 1)

    return _dijkstra(G, source, get_weight, cutoff=cutoff, target=target)


def clos_cen(G, u=None, distance=None, normalized=True):
    if distance is not None:
        # use Dijkstra's algorithm with specified attribute as edge weight 
        path_length = functools.partial(single_source_dijkstra_path_length_g,
                                        weight=distance)
    else:
        path_length = nx.single_source_shortest_path_length
    if u is None:
        nodes = [x for x in G.nodes() if nx.degree(G, x) > 3 ]
    else:
        nodes = [u]
    closeness_centrality = {}
    for n in nodes[len(nodes)/2:]:
        sp = path_length(G,n, target=nodes)
        totsp = sum(sp.values())
        if totsp > 0.0 and len(G) > 1:
            closeness_centrality[n] = (len(sp)-1.0) / totsp
            # normalize to number of nodes-1 in connected part
            if normalized:
                s = (len(sp)-1.0) / ( len(G) - 1 )
                closeness_centrality[n] *= s
        else:
            closeness_centrality[n] = 0.0
    if u is not None:
        return closeness_centrality[u]
    else:
        return closeness_centrality    
    
    
dcc = clos_cen(gfc, u=None, distance='dist')

import pickle
with open('dic_clos_cent3b.py', 'wb') as fp:
    pickle.dump(dcc, fp)


try:
    df = pd.DataFrame(dcc)
    df.to_csv('dic_clos_cent3b.csv')
except:
    pass




In [ ]:
### Closeness

import random as rn
import fiona
import pandas as pd
import numpy as np
from random import randint as rint
import networkx as nx
import functools

def dist(i,j):
    return np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 )



fp = fiona.open("/home/dario/trafico/redCDMX/VialidadOSM.shp")
calles = {}
siq  = [u'highway', u'barrier', u'id', u'nombre', u'osm_id', u'other_tags', u'sentido', u'tipo', u'z_order']
for feat in fp:
    calles[ feat['id'] ] = {}
    calles[ feat['id'] ]['type'] = feat['geometry']['type']
    calles[ feat['id'] ]['coords'] = feat['geometry']['coordinates']
    for key, val in feat['properties'].items():
        if key in siq:
            calles[ feat['id'] ][key] = val
            calles[ feat['id'] ][key] = val


fp.close()

ntwk = pd.DataFrame(calles).transpose()
ntwk.index = ntwk['id']
ntwk = ntwk.drop(['id'], axis=1)


i=0
GN = nx.DiGraph(weighted=True)
for row, col in ntwk.iterrows():
    p = ntwk.loc[row]['coords']
    lp = len(p)
    GN.add_path( p, name=ntwk.loc[row]['nombre'],
                vialidad = ntwk.loc[row]['highway'], osm_id = ntwk.loc[row]['osm_id'],
                sentido = ntwk.loc[row]['sentido'], tipo=ntwk.loc[row]['tipo']
               )
    i = lp + i
    
    
gnfc = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[0]
gfc = GN.subgraph(gnfc)

for edge in gfc.edges():
    gfc[edge[0]][edge[1]]['dist'] = dist(*edge)



def clos_cen(G, u=None, distance=None, normalized=True):
    if distance is not None:
        # use Dijkstra's algorithm with specified attribute as edge weight 
        path_length = functools.partial(nx.single_source_dijkstra_path_length,
                                        weight=distance)
    else:
        path_length = nx.single_source_shortest_path_length

    if u is None:
        nodes = [x for x in G.nodes() if nx.degree(G, x) > 3 ]
    else:
        nodes = [u]
    closeness_centrality = {}
    for n in nodes:
        sp = path_length(G,n)
        totsp = sum(sp.values())
        if totsp > 0.0 and len(G) > 1:
            closeness_centrality[n] = (len(sp)-1.0) / totsp
            # normalize to number of nodes-1 in connected part
            if normalized:
                s = (len(sp)-1.0) / ( len(G) - 1 )
                closeness_centrality[n] *= s
        else:
            closeness_centrality[n] = 0.0
    if u is not None:
        return closeness_centrality[u]
    else:
        return closeness_centrality    
    
    
dcc = clos_cen(gfc, u=None, distance='dist')

import pickle
with open('dic_clos_cent2.py', 'wb') as fp:
    pickle.dump(dcc, fp)


try:
    df = pd.DataFrame(dcc)
    df.to_csv('dic_clos_cent2.csv')
except:
    pass




In [ ]:
### ECCENTRICITY

import random as rn
import fiona
import pandas as pd
import numpy as np
from random import randint as rint
import networkx as nx
import functools

def dist(i,j):
    return np.sqrt( (i[0] - j[0])**2 + (i[1] - j[1])**2 )



fp = fiona.open("/home/dario/trafico/redCDMX/VialidadOSM.shp")
calles = {}
siq  = [u'highway', u'barrier', u'id', u'nombre', u'osm_id', u'other_tags', u'sentido', u'tipo', u'z_order']
for feat in fp:
    calles[ feat['id'] ] = {}
    calles[ feat['id'] ]['type'] = feat['geometry']['type']
    calles[ feat['id'] ]['coords'] = feat['geometry']['coordinates']
    for key, val in feat['properties'].items():
        if key in siq:
            calles[ feat['id'] ][key] = val
            calles[ feat['id'] ][key] = val


fp.close()

ntwk = pd.DataFrame(calles).transpose()
ntwk.index = ntwk['id']
ntwk = ntwk.drop(['id'], axis=1)


i=0
GN = nx.DiGraph(weighted=True)
for row, col in ntwk.iterrows():
    p = ntwk.loc[row]['coords']
    lp = len(p)
    GN.add_path( p, name=ntwk.loc[row]['nombre'],
                vialidad = ntwk.loc[row]['highway'], osm_id = ntwk.loc[row]['osm_id'],
                sentido = ntwk.loc[row]['sentido'], tipo=ntwk.loc[row]['tipo']
               )
    i = lp + i
    
    
gnfc = sorted(nx.strongly_connected_components(GN), key = len, reverse=True)[0]
gfc = GN.subgraph(gnfc)

for edge in gfc.edges():
    gfc[edge[0]][edge[1]]['dist'] = dist(*edge)



def eccen(G, v=None, distance=None,):
    order=G.order()
    e={}
    nodes = [x for x in G.nodes() if nx.degree(G, x) > 3 ]
    #for n in G.nbunch_iter(v):
    for n in nodes[:len(nodes)/2]:
        path_length = functools.partial(nx.single_source_dijkstra_path_length,weight=distance)
        length = path_length(G,n)    
        e[n]=max(length.values())
    if v in G:
        return e[v]  # return single value
    else:
        return e


decc = eccen(gfc, u=None, distance='dist')

import pickle
with open('dic_eccen_2.py', 'wb') as fp:
    pickle.dump(decc, fp)


try:
    df = pd.DataFrame(dcc)
    df.to_csv('dic_eccen_2.csv')
except:
    pass